![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [105]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [106]:
mod_time_occ = crimes["TIME OCC"].str.slice(stop=2)
peak_crime_hour = int(mod_time_occ.value_counts().index[0])
peak_crime_hour

12

In [107]:
crimes["TIME OCC DT"] = pd.to_datetime(crimes["TIME OCC"], format="%H%M")
index = pd.DatetimeIndex(crimes["TIME OCC DT"])
crimes_mod = crimes.copy().iloc[index.indexer_between_time("22:00","3:59")]
peak_night_crime_location = crimes_mod.groupby(by=["AREA NAME"]).count().sort_values(by=["DR_NO"], ascending=False).index[0]
peak_night_crime_location

'Central'

In [108]:
crimes_mod2 = crimes.copy()
crimes_mod2.loc[crimes_mod2["Vict Age"].between(0,17,inclusive="both"), "age_group"] = "0-17"
crimes_mod2.loc[crimes_mod2["Vict Age"].between(18,25,inclusive="both"), "age_group"] = "18-25"
crimes_mod2.loc[crimes_mod2["Vict Age"].between(26,34,inclusive="both"), "age_group"] = "26-34"
crimes_mod2.loc[crimes_mod2["Vict Age"].between(35,44,inclusive="both"), "age_group"] = "35-44"
crimes_mod2.loc[crimes_mod2["Vict Age"].between(45,54,inclusive="both"), "age_group"] = "45-54"
crimes_mod2.loc[crimes_mod2["Vict Age"].between(55,64,inclusive="both"), "age_group"] = "55-64"
crimes_mod2.loc[crimes_mod2["Vict Age"]>=65, "age_group"] = "65+"
crimes_mod2.index = crimes_mod2["age_group"]
crimes_mod2 = crimes_mod2.groupby(level=0).count()
crimes_mod2
# Enfoque erroneo

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION,TIME OCC DT,age_group
age_group,,,,,,,,,,,,,,
0-17,4528,4528,4528,4528,4528,4528,4528,4528,4528,3770,4528,4528,4528,4528
18-25,28291,28291,28291,28291,28291,28291,28291,28283,28285,12259,28291,28291,28291,28291
26-34,47470,47470,47470,47470,47470,47470,47470,47470,47470,18539,47470,47470,47470,47470
35-44,42157,42157,42157,42157,42157,42157,42157,42155,42155,16037,42157,42157,42157,42157
45-54,28353,28353,28353,28353,28353,28353,28353,28353,28353,10808,28353,28353,28353,28353
55-64,20169,20169,20169,20169,20169,20169,20169,20169,20168,7586,20169,20169,20169,20169
65+,14747,14747,14747,14747,14747,14747,14747,14746,14746,4503,14747,14747,14747,14747


In [109]:
victim_ages = crimes_mod2['age_group']
victim_ages

age_group
0-17      4528
18-25    28291
26-34    47470
35-44    42157
45-54    28353
55-64    20169
65+      14747
Name: age_group, dtype: int64